# Model fitting notebook
---


In [1]:
import matplotlib.pyplot as plt
from matplotlib.transforms import ScaledTranslation
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.special import logsumexp
from scipy.stats import norm
import scipy.optimize
import random
from src.functions import * 

# Set some defaults
plt.rc("axes.spines", top=False, right=False)
sns.set_theme(context="paper", font_scale=1.2)
sns.set_style("ticks")
plt.rc("axes.spines", top=False, right=False)

%config InlineBackend.figure_format = "retina"

---
## Load behavioral data


In [2]:
# Read in vmr data
df = pd.read_csv("../results/vmr_all.csv")
df.head()

,si,SN,id,tester,TN,move_cycle,hX,hY,rot_hX,rot_hY,...,RT,ST,adaptation,total error,theta_maxradv_z,theta_maxradv_outlier,theta_maxradv_mean,theta_maxradv_clean,motor_sd,perturbation
0,1,1,01_i,s,72,72,5.646301,90.778669,0.0,0.0,...,0.323414,1.054281,NaN,NaN,2.507849,False,1.076975,6.542682,1.661801,False
1,1,1,01_i,s,73,73,4.886301,88.328669,0.0,0.0,...,0.309482,1.084872,-2.337005,-2.972700,-0.007217,False,1.076975,1.027300,1.661801,True
2,1,1,01_i,s,74,74,5.006301,91.668669,0.0,0.0,...,0.356222,1.100153,NaN,NaN,1.442153,False,1.076975,4.205676,1.661801,False
3,1,1,01_i,s,75,75,2.246301,90.208669,0.0,0.0,...,0.276952,1.339061,-0.427851,1.467717,1.105636,False,1.076975,3.467717,1.661801,True
4,1,1,01_i,s,76,76,1.716301,87.748669,0.0,0.0,...,0.241043,1.486548,NaN,NaN,1.247048,False,1.076975,3.777826,1.661801,False


---
## Helper functions

In [3]:
def plot_individual(df, ege, ige, adaptation, color="k", title=None):
    fig, axs = plt.subplots(1, 2, figsize=(4.5, 3))
    sns.regplot(data=df, x=ege, x_jitter=0.1, y=adaptation, 
               scatter_kws={"alpha":0.15, "color":"gray"},  line_kws={"color":color}, ax=axs[0])
    axs[0].set(xlabel="EGE ($\degree$)", ylabel="Adaptation ($\degree$)", 
               xlim=[-6.5, 6.5], xticks=np.arange(-4, 4.01, 2), ylim=[-10, 10], 
               yticks=np.arange(-10, 10.01, 2))
    axs[0].axhline(color="k", linewidth=0.5)

    sns.regplot(data=df, x=ige, y=adaptation, line_kws={"color":color}, 
                scatter_kws={"alpha":0.4, "color":"gray"}, ax=axs[1])
    axs[1].set(ylim=[-10, 10], yticks=np.arange(-10, 10.01, 2), xlim=[-6.5, 6.5], 
               xticks=np.arange(-4, 4.01, 2), xlabel="IGE ($\degree$)", 
               ylabel="Adaptation ($\degree$)")
    axs[1].axhline(color="k", linewidth=0.5)
    sns.despine()
    fig.suptitle(title, y=0.90)
    plt.tight_layout()
    
    return fig


def plot_individual_ege(df, ege, ige, adaptation, color="k", ax=None, title=None):
    sns.regplot(data=df, x=ege, x_jitter=0.1, y=adaptation, 
               scatter_kws={"alpha":0.4, "color":"gray"},  line_kws={"color":color}, ax=ax)
    ax.set(xlabel="Rotation ($^\circ$)", ylabel="Adaptation  ($\degree$)", 
               xlim=[-6.5, 6.5], xticks=np.arange(-4, 4.01, 2), ylim=[-10, 10], 
               yticks=np.arange(-10, 10.01, 2))
    ax.axhline(color="k", linewidth=0.5)

def plot_individual_ige(df, ege, ige, adaptation, color="k", ax=None, title=None):
    sns.regplot(data=df, x=ige, y=adaptation, line_kws={"color":color}, 
                scatter_kws={"alpha":0.4, "color":"gray"}, ax=axs[1])
    axs[1].set(ylim=[-10, 10], yticks=np.arange(-10, 10.01, 2), xlim=[-6.5, 6.5], 
               xticks=np.arange(-4, 4.01, 2), xlabel="Motor noise ($\degree$)", 
               ylabel="Adaptation ($\degree$)")
    axs[1].axhline(color="k", linewidth=0.5)
    
    sns.despine()
    fig.suptitle(title)
    plt.tight_layout()
    
    return fig

---
## Fit data using Maximum Likelihoood Estimation (MLE)

In [4]:
%%time 
subj_num = []
model = []
theta = []
loglik = []
bic = []
delta_bic = []

# Fit models to data
for i in range(len(np.unique(df["SN"]))):
    subj = df.loc[df["SN"] == i + 1, :]
    
    # Extract important variables
    motor_sd = subj["motor_sd"].iloc[0]
    x_hand = subj["theta_maxradv_clean"].values
    rotation = subj["rotation"].values
    vis_fb = subj["fbi"].values

    # Define cost functions
    nll_pea = lambda x: negloglik(model="pea", sigma_int=x[0], B=x[1], bias=x[2], 
                                       sigma_motor=motor_sd, num_trials=len(subj), 
                                       vis_fb=vis_fb, rotation=rotation, x_hand=x_hand)
    nll_premo = lambda x: negloglik(model="premo", B=x[0], sigma_pred=x[1], 
                                         sigma_v=x[2], sigma_p=x[3], eta_p=x[4], bias=x[5], 
                                         sigma_motor=motor_sd, num_trials=len(subj), 
                                         vis_fb=vis_fb, rotation=rotation, 
                                         x_hand=x_hand)
    nll_piece = lambda x: negloglik(model="piece", sigma_pert=x[0], sigma_comb=x[1], 
                                         bias = x[2], sigma_motor=motor_sd, num_trials=len(subj), 
                                         vis_fb=vis_fb, rotation=rotation, x_hand=x_hand)
    nll_piece_variant = lambda x: negloglik(model="piece_variant", sigma_pert=x[0], sigma_comb=x[1], 
                                         bias = x[2], sigma_motor=motor_sd, num_trials=len(subj), 
                                         vis_fb=vis_fb, rotation=rotation, x_hand=x_hand)
    nll_rem = lambda x: negloglik(model="rem", sigma_comb=x[0], s=x[1], c=x[2], bias=x[3],
                                       sigma_motor=motor_sd, num_trials=len(subj), 
                                       vis_fb=vis_fb, rotation=rotation, x_hand=x_hand)
    
    # Fit models: use different random initializations
    # for k in range(10):
    for k in range(3):
        # PEA model params: sigma_comb, B, bias
        bounds = ((0.01, 25), (0, 1), (-5, 5))
        tmp_pea = scipy.optimize.minimize(
            fun=nll_pea, 
            bounds=bounds,
            x0=np.array([np.random.uniform(low=bounds[0][0], high=bounds[0][1]),
                         np.random.uniform(low=bounds[1][0], high=bounds[1][1]),
                         np.random.uniform(low=bounds[2][0], high=bounds[2][1])])
        )
            
        # PReMo model params: B, sigma_v, sigma_p, sigma_pred, eta_p, bias
        bounds = ((0, 1), (0.01, 25), (0.01, 25), (0.01, 25), (0, 1), (-5, 5))
        tmp_premo = scipy.optimize.minimize(
            fun=nll_premo,  
            bounds=bounds,
            x0=np.array([np.random.uniform(low=bounds[0][0], high=bounds[0][1]),
                         np.random.uniform(low=bounds[1][0], high=bounds[1][1]),
                         np.random.uniform(low=bounds[2][0], high=bounds[2][1]),
                         np.random.uniform(low=bounds[3][0], high=bounds[3][1]),
                         np.random.uniform(low=bounds[4][0], high=bounds[4][1]),
                         np.random.uniform(low=bounds[5][0], high=bounds[5][1])])
        )
    
        # REM model params: sigma_comb, s, c, bias
        bounds = ((0.01, 25), (0, 10), (0, 10), (-5, 5))
        tmp_rem = scipy.optimize.minimize(
            fun=nll_rem, 
            bounds=bounds,
            x0=np.array([np.random.uniform(low=bounds[0][0], high=bounds[0][1]),
                         np.random.uniform(low=bounds[1][0], high=bounds[1][1]),
                         np.random.uniform(low=bounds[2][0], high=bounds[2][1]),
                         np.random.uniform(low=bounds[3][0], high=bounds[3][1])])
        )
    
        # PIECE model params: sigma_pert, sigma_comb, bias
        # bounds = ((0.01, 30), (0.01, 25), (-5, 5))
        bounds = ((0.01, 30), (0.2, 25), (-5, 5))  # sigma_comb: 10 x's < motor var
        tmp_piece = scipy.optimize.minimize(
            fun=nll_piece, 
            bounds=bounds,
            x0=np.array([np.random.uniform(low=bounds[0][0], high=bounds[0][1]),
                         np.random.uniform(low=bounds[1][0], high=bounds[1][1]),
                         np.random.uniform(low=bounds[2][0], high=bounds[2][1])])
        )

        # PIECE VARIANT model params: sigma_pert, sigma_comb, bias
        bounds = ((0.01, 30), (0.2, 25), (-5, 5))  # sigma_comb: 10 x's < motor var
        tmp_piece_variant = scipy.optimize.minimize(
            fun=nll_piece_variant, 
            bounds=bounds,
            x0=np.array([np.random.uniform(low=bounds[0][0], high=bounds[0][1]),
                         np.random.uniform(low=bounds[1][0], high=bounds[1][1]),
                         np.random.uniform(low=bounds[2][0], high=bounds[2][1])])
        )

        # Compare negative log-likelihoods; assign best results
        if k == 0:
            pea_results = tmp_pea
            premo_results = tmp_premo
            rem_results = tmp_rem
            piece_results = tmp_piece
            piece_variant_results = tmp_piece_variant
        elif k != 0: 
            if tmp_pea.fun < pea_results.fun:
                pea_results = tmp_pea
            if tmp_premo.fun < premo_results.fun:
                premo_results = tmp_premo
            if tmp_rem.fun < rem_results.fun:
                rem_results = tmp_rem
            if tmp_piece.fun < piece_results.fun:
                piece_results = tmp_piece
            if tmp_piece_variant.fun < piece_variant_results.fun:
                piece_varaint_results = tmp_piece_variant
            
        print(f"{i: } {pea_results.message}")
        print(f"{i: } {premo_results.message}")
        print(f"{i: } {rem_results.message}") 
        print(f"{i: } {piece_results.message}")
        print(f"{i: } {piece_variant_results.message}")
    
    # Fill in variables for "fits" data frame
    subj_num.extend([i + 1, i + 1, i + 1, i + 1, i + 1])
    model.extend(["pea", "premo", "rem", "piece", "piece_variant"])
    theta.extend([pea_results.x, premo_results.x, rem_results.x, 
                  piece_results.x, piece_variant_results.x])
    loglik.extend([pea_results.fun * -1, 
                   premo_results.fun * -1, rem_results.fun * -1, piece_results.fun * -1,
                   piece_variant_results.fun * -1])
    bic.extend([calc_bic(pea_results.fun * -1, len(pea_results.x), len(subj)),
                calc_bic(premo_results.fun * -1, len(premo_results.x), len(subj)),
                calc_bic(rem_results.fun * -1, len(rem_results.x), len(subj)),
                calc_bic(piece_results.fun * -1, len(piece_results.x), len(subj)),
                calc_bic(piece_variant_results.fun * -1, len(piece_variant_results.x), len(subj))])
    bic_ref = calc_bic(premo_results.fun * -1, len(premo_results.x), len(subj))  # premo serves as benchmark
    delta_bic.extend([bic[-5] - bic_ref, bic[-4] - bic_ref, bic[-3] - bic_ref, bic[-2] - bic_ref, bic[-1] - bic_ref])

fits = pd.DataFrame({
    "subj_num":subj_num, 
    "model":model, 
    "theta":theta, 
    "loglik":loglik, 
    "bic":bic,
    "delta_bic":delta_bic
})

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
 0 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 0 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 0 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 0 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 0 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 0 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 0 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 0 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 0 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 0 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 0 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 0 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 0 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 0 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 0 CONVERGENCE: REL_REDUCTION_OF_F_<

/Users/hyosubkim/Documents/GitHub/Projects/ige-ege/src/functions.py:164: RuntimeWarning: invalid value encountered in double_scalars
  p_rel = s * (np.exp(-(x_v)**2 / (2 * sigma_comb**2))) / (np.exp(-(x_v)**2 / (2 * sigma_comb**2)) + c)


 2 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 2 CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
 2 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 2 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 2 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 2 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 2 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 2 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 2 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 2 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 2 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 2 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 2 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 2 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 2 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 3 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 3 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 3 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 3 CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
 3 CONVERGENCE: REL_REDUCTION_

In [10]:
fits.groupby("model")["bic"].describe().reset_index()

,model,count,mean,std,min,25%,50%,75%,max
0,pea,16.0,5484.910635,496.624144,4653.087331,5229.327725,5426.178122,5731.352489,6742.904940
1,piece,16.0,5287.453007,491.713270,4403.888481,5041.291068,5240.772171,5498.866328,6439.587150
2,piece_variant,16.0,5293.710538,494.137424,4403.888481,5045.019479,5265.423672,5483.351972,6475.675429
3,premo,16.0,5609.612951,477.301441,4945.510690,5281.779421,5519.929286,5913.855521,6810.257159
4,rem,16.0,5496.215971,493.425316,4659.838929,5234.368612,5433.095718,5738.606708,6741.774732


---
## Read and/or write parameters

In [ ]:
# Save MLEs? 
write_data = False

# Read-in saved parameters?
read_data = True

# Helper function
def converter(input_str):
    return np.fromstring(input_str[1:-1], sep=' ')
    
# Write data to csv; look into preserving numpy dtype for "theta" column
if write_data:
    # fits.to_csv("../results/params_mle_reducedmodels.csv", index=False)
    fits.to_csv("../results/params_mle_higher_sigmau.csv", index=False)
# Need a function to convert string back to numpy array
if read_data:
    # fits = pd.read_csv("../results/params_mle_reducedmodels.csv", converters={"theta":converter})
    # fits = pd.read_csv("../results/params_mle_higher_sigmau.csv", converters={"theta":converter})
    fits = pd.read_csv("../results/params_mle_piecevariant.csv", converters={"theta":converter})
fits.head()

In [ ]:
# Plot MLEs for PIECE params
sigma_u = np.zeros(16)
sigma_comb = np.zeros(16)
for i in range(16):
    for j in range(2):
        if j == 0:
            sigma_u[i] = fits.loc[fits["model"] == "piece", "theta"].iloc[i][j]
        elif j == 1:
            sigma_comb[i] = fits.loc[fits["model"] == "piece", "theta"].iloc[i][j]

idx_outlier = sigma_comb.argmax() # Crazy value of 25
sigma_u = np.delete(sigma_u, idx_outlier)
sigma_comb = np.delete(sigma_comb, idx_outlier)
sigma_comb

plt.rc("axes.spines", top=False, right=False)

# Plot MLEs of PIECE params
param_vals = np.hstack((sigma_u, sigma_comb))
param_labels = []
[param_labels.append("sigma_u") for x in range(len(sigma_u))] 
[param_labels.append("sigma_comb") for x in range(len(sigma_comb))]
piece_mle = pd.DataFrame({"param_labels":param_labels, "param_vals":param_vals})
piece_mle.head()

fig, ax = plt.subplots(figsize=(3, 3))
sns.pointplot(data=piece_mle, x="param_labels", y="param_vals", ax=ax, c="b", linestyle="none")
sns.stripplot(data=piece_mle, x="param_labels", y="param_vals", alpha=0.3, ax=ax, c="b")
ax.set(xlabel=None, xticklabels=["$\sigma_r$", "$\sigma_{combined}$"], ylabel="MLE")
plt.tight_layout()
plt.show()
fig.savefig("images/param_mles.png", dpi=300)

In [ ]:
# Compute BIC deltas between models
idx_pea = fits["model"] == "pea"
idx_piece = fits["model"] == "piece"
idx_rem = fits["model"] == "rem"
idx_premo = fits["model"] == "premo"

# fits["piece_v_pea"] = np.zeros(len(fits))
diff_pea = fits.loc[idx_piece, "bic"].values - fits.loc[idx_pea, "bic"].values
diff_premo = fits.loc[idx_piece, "bic"].values - fits.loc[idx_premo, "bic"].values
diff_rem = fits.loc[idx_piece, "bic"].values - fits.loc[idx_rem, "bic"].values

fits["piece_v_pea"] = np.repeat(diff_pea, 4)  # repeating to fill appropriate rows
fits["piece_v_premo"] = np.repeat(diff_premo, 4)
fits["piece_v_rem"] = np.repeat(diff_rem, 4)
fits

In [ ]:
mask = fits["model"] == "piece"
fits.loc[mask, "theta"]


In [ ]:
# Assign colors to models
colors = {"premo":"tab:green", "piece":"tab:purple",
          "pea":"tab:orange", "rem":"tab:red"}

# Plot BIC comparisons
fig_bic, axs = plt.subplots(2, 2,figsize=(7.25, 6))
sns.despine()
sns.stripplot(
    data=fits, x="model", y="delta_bic", dodge=False, alpha=0.3, hue="model", 
    legend=False, order=["premo", "piece", "pea", "rem"] , palette=colors,
        ax=axs[0, 0]
)
sns.pointplot(
    data=fits, x="model", y="delta_bic", hue="model", dodge=0.0,
    order=["premo", "piece", "pea", "rem"], **{"linestyle":"none"},
    legend=False, ax=axs[0, 0], palette=colors
)
axs[0, 0].axhline(y=0, color="k", linewidth=0.5)
axs[0, 0].set(ylabel=r"$\Delta$BIC", xticklabels=["PReMo", "PIECE", "PEA", "REM"], 
              xlabel="Model", ylim=(-600, 50))
axs[0, 0].text(-1.2, 65, "Worse", fontsize="small")
axs[0, 0].text(-1.2, -675, "Better", fontsize="small")
axs[0, 0].text(-0.35, 1.15, "a", transform=axs[0, 0].transAxes, 
               fontsize=14, weight="bold")

# PIECE vs PReMo
sns.barplot(
    data=fits, x="subj_num", y="piece_v_premo", errorbar=None, color="tab:purple", 
    ax=axs[0, 1]
)
axs[0, 1].axhline(c="k", linewidth=1)
axs[0, 1].set(xlabel="Participant", ylabel=r"$\Delta$BIC (PIECE - PReMo)", ylim=(-600, 50))
axs[0, 1].tick_params(axis="x", labelsize=9)
axs[0, 1].text(-6, 66, "Favors PReMo", fontsize="small")
axs[0, 1].text(-6.25, -675, "Favors PIECE", fontsize="small")
axs[0, 1].text(-0.35, 1.15, "b", transform=axs[0, 1].transAxes, 
               fontsize=14, weight="bold")

# PIECE vs PEA
sns.barplot(
    data=fits, x="subj_num", y="piece_v_pea", errorbar=None, color="tab:purple", 
    ax=axs[1, 0]
)
axs[1, 0].axhline(c="k", linewidth=1)
axs[1, 0].set(xlabel="Participant", ylabel=r"$\Delta$BIC (PIECE - PEA)", ylim=(-600, 50))
axs[1, 0].tick_params(axis="x", labelsize=9)
axs[1, 0].text(-6, 66, "Favors PEA", fontsize="small")
axs[1, 0].text(-6.25, -675, "Favors PIECE", fontsize="small")
axs[1, 0].text(-0.35, 1.15, "c", transform=axs[1, 0].transAxes, 
               fontsize=14, weight="bold")

# PIECE vs REM
sns.barplot(
    data=fits, x="subj_num", y="piece_v_rem", errorbar=None, color="tab:purple", 
    ax=axs[1, 1]
)
axs[1, 1].axhline(c="k", linewidth=1)
axs[1, 1].set(xlabel="Participant", ylabel=r"$\Delta$BIC (PIECE - REM)", ylim=(-600, 50))
axs[1, 1].tick_params(axis="x", labelsize=9)
axs[1, 1].text(-6, 66, "Favors REM", fontsize="small")
axs[1, 1].text(-6.25, -675, "Favors PIECE", fontsize="small")
axs[1, 1].text(-0.35, 1.15, "d", transform=axs[1, 1].transAxes, 
               fontsize=14, weight="bold")
plt.tight_layout()


In [ ]:
# fig_bic.savefig("images/bic.pdf", dpi=300)

---
# Plot Models using MLEs ($\hat{\Theta}_{\text{MLE}}$) 

In [ ]:
# Choose a sample subject
sid = 14
sample_subj = df.loc[df["SN"] == sid, :].reset_index(drop=True)
sigma_motor = sample_subj.loc[0, "motor_sd"]

# Pull out best-fit params for chosen subject:
params_idx = fits["subj_num"] == sid 
params = fits.loc[params_idx, ["theta", "model"]].iloc[0:5, :].reset_index(drop=True)

# Pull out MLEs for each model
pea_mle = params.loc[params["model"] == "pea", "theta"].values[0]
premo_mle = params.loc[params["model"] == "premo", "theta"].values[0]
piece_mle = params.loc[params["model"] == "piece", "theta"].values[0]
rem_mle = params.loc[params["model"] == "rem", "theta"].values[0]

# Find perturbation trials
pert_indices = np.flatnonzero(sample_subj["perturbation"])

# Simulate with best-fit parameters
# PIECE model
_, xhat_piece = piece(
    piece_mle[0], 
    piece_mle[1], 
    piece_mle[2],
    sigma_motor, len(sample_subj), 
    sample_subj["fbi"], 
    sample_subj["rotation"],
    fit=False
)
adapt_piece = xhat_piece[pert_indices + 1] - xhat_piece[pert_indices - 1]

# PEA model
_, xhat_pea = pea(
    pea_mle[0], 
    pea_mle[1],
    pea_mle[2],
    sigma_motor, len(sample_subj), 
    sample_subj["fbi"], 
    sample_subj["rotation"],
    fit=False
)
adapt_pea = xhat_pea[pert_indices + 1] - xhat_pea[pert_indices - 1]

# PReMo 
_, xhat_premo = premo(
    premo_mle[0],
    premo_mle[1],
    premo_mle[2], 
    premo_mle[3],
    premo_mle[4],
    premo_mle[5],
    sigma_motor, len(sample_subj), 
    sample_subj["fbi"], 
    sample_subj["rotation"],
    fit=False
)
adapt_premo = xhat_premo[pert_indices + 1] - xhat_premo[pert_indices - 1]

# REM
_, xhat_rem = rem(
    rem_mle[0],
    rem_mle[1], 
    rem_mle[2],
    rem_mle[3],
    sigma_motor, len(sample_subj), 
    sample_subj["fbi"], 
    sample_subj["rotation"],
    fit=False
)
adapt_rem = xhat_rem[pert_indices + 1] - xhat_rem[pert_indices - 1]

# Create data frame with all perturbation trials
df1 = pd.DataFrame({
    "rotation":sample_subj.loc[pert_indices, "rotation"],
    "xhat_piece":xhat_piece[pert_indices],
    "adapt_piece":adapt_piece,
    "xhat_pea":xhat_pea[pert_indices],
    "adapt_pea":adapt_pea,
    "xhat_premo":xhat_premo[pert_indices],
    "adapt_premo":adapt_premo,
    "xhat_rem":xhat_rem[pert_indices], 
    "adapt_rem":adapt_rem
})

# Trim data frame to adaptation only trials
sample_subj = sample_subj.loc[pert_indices, :]


In [ ]:
# Assign colors to models
premo_color = "tab:green"
piece_color = "tab:purple"
pea_color = "tab:orange"
rem_color = "tab:red"

# Plot best-fits
fig1 = plot_individual(sample_subj, "rotation", "theta_maxradv_clean", "adaptation", title="Data")
plt.gcf().text(0.02, 0.92, "a", fontsize=14, weight="bold")
slope_ege_data, _, _, _, _ = stats.linregress(sample_subj["rotation"], sample_subj["adaptation"])
slope_ige_data, _, _, _, _ = stats.linregress(sample_subj["theta_maxradv_clean"], sample_subj["adaptation"])

fig2 = plot_individual(df1, "rotation", "xhat_piece", "adapt_piece", piece_color, r"PIECE")
plt.gcf().text(0.02, 0.92, "b", fontsize=14, weight="bold")
slope_ege_piece, _, _, _, _ = stats.linregress(df1["rotation"], df1["adapt_piece"])
slope_ige_piece, _, _, _, _ = stats.linregress(df1["xhat_piece"], df1["adapt_piece"])

fig3 = plot_individual(df1, "rotation", "xhat_premo", "adapt_premo", premo_color, r"PReMo")
plt.gcf().text(0.02, 0.92, "c", fontsize=14, weight="bold")
slope_ege_premo, _, _, _, _ = stats.linregress(df1["rotation"], df1["adapt_premo"])
slope_ige_premo, _, _, _, _ = stats.linregress(df1["xhat_premo"], df1["adapt_premo"])

fig4 = plot_individual(df1, "rotation", "xhat_pea", "adapt_pea", pea_color, r"PEA")
plt.gcf().text(0.02, 0.92, "d", fontsize=14, weight="bold")
slope_ege_pea, _, _, _, _ = stats.linregress(df1["rotation"], df1["adapt_pea"])
slope_ige_pea, _, _, _, _ = stats.linregress(df1["xhat_pea"], df1["adapt_pea"])

fig5 = plot_individual(df1, "rotation", "xhat_rem", "adapt_rem", rem_color, r"REM")
plt.gcf().text(0.02, 0.92, "e", fontsize=14, weight="bold")
slope_ege_rem, _, _, _, _ = stats.linregress(df1["rotation"], df1["adapt_rem"])
slope_ige_rem, _, _, _, _ = stats.linregress(df1["xhat_piece"], df1["adapt_rem"])
plt.show()

In [ ]:
print(f"slope ege data: {slope_ege_data:.2f}")
print(f"slope ige data: {slope_ige_data:.2f}")
print(f"slope ege piece: {slope_ege_piece:.2f}")
print(f"slope ige piece: {slope_ige_piece:.2f}")
print(f"slope ege premo: {slope_ege_premo:.2f}")
print(f"slope ige premo: {slope_ige_premo:.2f}")
print(f"slope ege pea: {slope_ege_pea:.2f}")
print(f"slope ige pea: {slope_ige_pea:.2f}")
print(f"slope ege rem: {slope_ege_rem:.2f}")
print(f"slope ige rem: {slope_ige_rem:.2f}")

In [ ]:
# fig1.savefig("images/s13-data.pdf", dpi=300)
# fig2.savefig("images/s13-postpred-piece.pdf", dpi=300)
# fig3.savefig("images/s13-postpred-premo.pdf", dpi=300)
# fig4.savefig("images/s13-postpred-pea.pdf", dpi=300)
# fig5.savefig("images/s13-postpred-rem.pdf", dpi=300)